# Part III
Using the previous two tutorials, please answer the following using an encorder-decoder approach and an LSTM compared approach.

Please create a transformer-based classifier for English name classification into male or female.

There are several datasets for name for male or female classification. In subseuqent iterations, this could be expanded to included more classifications.

Below is the source from NLTK, which only has male and female available but could be used for the purposes of this assignment.

```
names = nltk.corpus.names
names.fileids()
['female.txt', 'male.txt']
male_names = names.words('male.txt')
female_names = names.words('female.txt')
[w for w in male_names if w in female_names]
['Abbey', 'Abbie', 'Abby', 'Addie', 'Adrian', 'Adrien', 'Ajay', 'Alex', 'Alexis',
'Alfie', 'Ali', 'Alix', 'Allie', 'Allyn', 'Andie', 'Andrea', 'Andy', 'Angel',
'Angie', 'Ariel', 'Ashley', 'Aubrey', 'Augustine', 'Austin', 'Averil', ...]
```

In [ ]:
### START CODE HERE (REPLACE INSTANCES OF 'None' with your code) ###
!pip install --upgrade keras
!pip install tensorflow

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 9.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.2/311.2 kB 10.5 MB/s eta 0:00:00
  Attempting uninstall: keras
    Found existing installation: keras 2.15.0
    Uninstalling keras-2.15.0:
      Successfully uninstalled keras-2.15.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.15.0 requires keras<2.16,>=2.15.0, but you have keras 3.3.2 which is incompatible.


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 11.2 MB/s eta 0:00:00
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/pip/_vendor/pkg_resources/__init__.py", line 3108, in _dep_map
    return self.__dep_map
  File "/usr/local/lib/python3.10/dist-packages/pip/_vendor/pkg_resources/__init__.py", line 2901, in __getattr__
    raise AttributeError(attr)
AttributeError: _DistInfoDistribution__dep_map

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/pip/_internal/cli/base_command.py", line 169, in exc_logging_wrapper
    status = run_func(*args)
  File "/usr/local/lib/python3.10/dist-packages/pip/_internal/cli/req_command.py", line 242, in wrapper
    return func(self, options, args)
  File "/usr/local/lib/python3.10/dist-packages/pip/_internal/commands/install.py", line 441, in run
    conflicts = self._determine_conflicts(to_install)
  File "/u

In [ ]:
import numpy as np
import tensorflow as tf
import keras
import string
import matplotlib.pyplot as plt
from keras import ops
from keras import layers
import random
import re
import nltk

## Downloading dataset

In [ ]:
nltk.download('names')

[nltk_data] Downloading package names to /root/nltk_data...
[nltk_data]   Unzipping corpora/names.zip.


True

In [ ]:
names=nltk.corpus.names

In [ ]:
names.fileids()

['female.txt', 'male.txt']

In [ ]:
male_names=names.words('male.txt')

In [ ]:
male_names[:10]

['Aamir',
 'Aaron',
 'Abbey',
 'Abbie',
 'Abbot',
 'Abbott',
 'Abby',
 'Abdel',
 'Abdul',
 'Abdulkarim']

In [ ]:
female_names=names.words('female.txt')
female_names[:10]

['Abagael',
 'Abagail',
 'Abbe',
 'Abbey',
 'Abbi',
 'Abbie',
 'Abby',
 'Abigael',
 'Abigail',
 'Abigale']

In [ ]:
max_length_name=max(len(name) for name in male_names)
print(max_length_name)

15


In [ ]:
max_length_name=max(len(name) for name in female_names)
print(max_length_name)

15


In [ ]:
max_len=16

In [ ]:
#defining vocab as letters
vocab_size=27
letters=string.ascii_lowercase
letters

'abcdefghijklmnopqrstuvwxyz'

In [ ]:
X=[]
y=[]

for name in male_names:
  name = re.sub(r'[^A-Za-z]', '', name)
  name_letters = list(name.lower())
  x_name = []
  for letter in name_letters:
      x_name.append(letters.index(letter)+1)
  X.append(x_name)
  y.append(1)

for name in female_names:
  name = re.sub(r'[^A-Za-z]', '', name)
  name_letters = list(name.lower())
  x_name = []
  for letter in name_letters:
      x_name.append(letters.index(letter)+1)
  X.append(x_name)
  y.append(0)



In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.20, random_state=42)

In [ ]:
len(X_train)

6355

In [ ]:
len(X_test)

1589

In [ ]:
X_train = keras.utils.pad_sequences(X_train, maxlen=max_len)
X_test = keras.utils.pad_sequences(X_test, maxlen=max_len)
y_train = np.array(y_train)
y_test = np.array(y_test)

In [ ]:
class TransformerBlock(layers.Layer):
    def __init__(self, embed_dim, num_heads, ff_dim, rate=0.1):
        super().__init__()
        self.att = layers.MultiHeadAttention(num_heads=num_heads, key_dim=embed_dim)
        self.ffn = keras.Sequential(
            [layers.Dense(ff_dim, activation="relu"), layers.Dense(embed_dim),]
        )
        self.layernorm1 = layers.LayerNormalization(epsilon=1e-6)
        self.layernorm2 = layers.LayerNormalization(epsilon=1e-6)
        self.dropout1 = layers.Dropout(rate)
        self.dropout2 = layers.Dropout(rate)

    def call(self, inputs):
        attn_output = self.att(inputs, inputs)
        attn_output = self.dropout1(attn_output)
        out1 = self.layernorm1(inputs + attn_output)
        ffn_output = self.ffn(out1)
        ffn_output = self.dropout2(ffn_output)
        return self.layernorm2(out1 + ffn_output)

class TokenAndPositionEmbedding(layers.Layer):
    def __init__(self, maxlen, vocab_size, embed_dim):
        super().__init__()
        self.token_emb = layers.Embedding(input_dim=vocab_size, output_dim=embed_dim)
        self.pos_emb = layers.Embedding(input_dim=maxlen, output_dim=embed_dim)

    def call(self, x):
        maxlen = ops.shape(x)[-1]
        positions = ops.arange(start=0, stop=maxlen, step=1)
        positions = self.pos_emb(positions)
        x = self.token_emb(x)
        return x + positions


In [ ]:
embed_dim =  64
num_heads = 3
ff_dim = 64
inputs = layers.Input(shape=(max_len,))
embedding_layer = TokenAndPositionEmbedding(max_len, vocab_size, embed_dim)
x = embedding_layer(inputs)
transformer_block = TransformerBlock(embed_dim, num_heads, ff_dim)
x = transformer_block(x)
x = layers.GlobalAveragePooling1D()(x)
x = layers.Dropout(0.1)(x)
x = layers.Dense(20, activation="leaky_relu")(x)
x = layers.Dropout(0.1)(x)
outputs = layers.Dense(2, activation="softmax")(x)

model = keras.Model(inputs=inputs, outputs=outputs)

In [ ]:
model.compile(optimizer="adam", loss="sparse_categorical_crossentropy", metrics=["accuracy"])
history = model.fit( X_train, y_train,
                     batch_size=32, epochs=25,
                     validation_data=(X_test, y_test) )


Epoch 1/25
199/199 ━━━━━━━━━━━━━━━━━━━━ 9s 25ms/step - accuracy: 0.6383 - loss: 0.6323 - val_accuracy: 0.7552 - val_loss: 0.4957
Epoch 2/25
199/199 ━━━━━━━━━━━━━━━━━━━━ 4s 18ms/step - accuracy: 0.7664 - loss: 0.4821 - val_accuracy: 0.7061 - val_loss: 0.5427
Epoch 3/25
199/199 ━━━━━━━━━━━━━━━━━━━━ 4s 18ms/step - accuracy: 0.7763 - loss: 0.4575 - val_accuracy: 0.7885 - val_loss: 0.4371
Epoch 4/25
199/199 ━━━━━━━━━━━━━━━━━━━━ 5s 24ms/step - accuracy: 0.7895 - loss: 0.4337 - val_accuracy: 0.7948 - val_loss: 0.4282
Epoch 5/25
199/199 ━━━━━━━━━━━━━━━━━━━━ 4s 19ms/step - accuracy: 0.7908 - loss: 0.4335 - val_accuracy: 0.7923 - val_loss: 0.4297
Epoch 6/25
199/199 ━━━━━━━━━━━━━━━━━━━━ 6s 21ms/step - accuracy: 0.8036 - loss: 0.4135 - val_accuracy: 0.7948 - val_loss: 0.4313
Epoch 7/25
199/199 ━━━━━━━━━━━━━━━━━━━━ 5s 25ms/step - accuracy: 0.7922 - loss: 0.4251 - val_accuracy: 0.8018 - val_loss: 0.4208
Epoch 8/25
199/199 ━━━━━━━━━━━━━━━━━━━━ 4s 21ms/step - accuracy: 0.7984 - loss: 0.4101 - val_accu

In [ ]:
#LSTM

## LSTM

In [ ]:
embed_dim =  64
inputs = layers.Input(shape=(max_len,))
embedding_layer = layers.Embedding(input_dim=vocab_size, output_dim=embed_dim)
x = embedding_layer(inputs)
x = layers.LSTM(64, return_sequences=True)(x)
x = layers.LSTM(32)(x)
x = layers.Dropout(0.1)(x)
x = layers.Dense(20, activation="leaky_relu")(x)
x = layers.Dropout(0.1)(x)
outputs = layers.Dense(2, activation="softmax")(x)

model_lstm = keras.Model(inputs=inputs, outputs=outputs)

In [ ]:
model_lstm.compile(optimizer="adam", loss="sparse_categorical_crossentropy", metrics=["accuracy"])
history = model_lstm.fit( X_train, y_train,
                          batch_size=32, epochs=15,
                          validation_data=(X_test, y_test) )

Epoch 1/15
199/199 ━━━━━━━━━━━━━━━━━━━━ 67s 30ms/step - accuracy: 0.6615 - loss: 0.6312 - val_accuracy: 0.7778 - val_loss: 0.4609
Epoch 2/15
199/199 ━━━━━━━━━━━━━━━━━━━━ 4s 22ms/step - accuracy: 0.7794 - loss: 0.4567 - val_accuracy: 0.7734 - val_loss: 0.4422
Epoch 3/15
199/199 ━━━━━━━━━━━━━━━━━━━━ 5s 23ms/step - accuracy: 0.8002 - loss: 0.4282 - val_accuracy: 0.7829 - val_loss: 0.4259
Epoch 4/15
199/199 ━━━━━━━━━━━━━━━━━━━━ 6s 25ms/step - accuracy: 0.7939 - loss: 0.4321 - val_accuracy: 0.7980 - val_loss: 0.4251
Epoch 5/15
199/199 ━━━━━━━━━━━━━━━━━━━━ 4s 22ms/step - accuracy: 0.8013 - loss: 0.4098 - val_accuracy: 0.7974 - val_loss: 0.4152
Epoch 6/15
199/199 ━━━━━━━━━━━━━━━━━━━━ 6s 28ms/step - accuracy: 0.8069 - loss: 0.4044 - val_accuracy: 0.7999 - val_loss: 0.4074
Epoch 7/15
199/199 ━━━━━━━━━━━━━━━━━━━━ 4s 21ms/step - accuracy: 0.8101 - loss: 0.4010 - val_accuracy: 0.8099 - val_loss: 0.4099
Epoch 8/15
199/199 ━━━━━━━━━━━━━━━━━━━━ 5s 26ms/step - accuracy: 0.8088 - loss: 0.3993 - val_acc

In [ ]:
def encode_name(name, maxlen=max_len):
    name = re.sub(r'[^A-Za-z]', '', name)
    name_letters = list(name.lower())
    x_name = []
    for letter in name_letters:
        #print(letter)
        x_name.append(letters.index(letter)+1)
    return keras.utils.pad_sequences([x_name], maxlen=max_len)

In [ ]:
test_names = []
from random import sample
sample_male_names=sample(male_names,4)
sample_female_names=sample(female_names,4)
test_names=sample_male_names+sample_female_names
test_names

['Mohammed',
 'Socrates',
 'Martino',
 'Remus',
 'Shaina',
 'Shannon',
 'Fanechka',
 'Ursola']

In [ ]:
print('Name', "    \t",   "Transformer", "\t" ,   "LSTM" )
print("===============================================")
for name in test_names:
    predicted_transformer = model.predict(encode_name(name), verbose=0)
    predicted_lstm  = model_lstm.predict(encode_name(name), verbose=0)
    print(name, "    \t",   "male" if np.argmax(predicted_transformer) == 1 else "female", "    \t" ,  "male" if np.argmax(predicted_lstm) == 1 else "female" )
    print("______________________________________________")

Name     	 Transformer 	 LSTM
Mohammed     	 male     	 male
______________________________________________
Socrates     	 female     	 male
______________________________________________
Martino     	 male     	 male
______________________________________________
Remus     	 male     	 male
______________________________________________
Shaina     	 female     	 female
______________________________________________
Shannon     	 male     	 male
______________________________________________
Fanechka     	 female     	 female
______________________________________________
Ursola     	 female     	 female
______________________________________________
